## Concat

In [1]:
import pandas as pd
import os

# 폴더 내 CSV 파일을 읽고 하나로 합치는 함수
def read_csv_files_in_folder(folder_path):
    all_files = []
    for file in os.listdir(folder_path):
        if file.endswith('.csv'):
            file_path = os.path.join(folder_path, file)
            try:
                df = pd.read_csv(file_path, encoding='utf-8')  # 먼저 utf-8로 시도
            except UnicodeDecodeError:
                try:
                    df = pd.read_csv(file_path, encoding='cp949')  # 한글 파일은 cp949도 자주 사용
                except Exception as e:
                    print(f"파일 {file}을(를) 읽는 데 실패했습니다: {e}")
                    continue
            all_files.append(df)
    
    combined_df = pd.concat(all_files, ignore_index=True)
    return combined_df

In [8]:
water_dispenser_path = './water_dispenser'
water_dispenser_combined_df = read_csv_files_in_folder(water_dispenser_path)

len(water_dispenser_combined_df)

1625

In [9]:
water_dispenser_combined_df.to_csv('./water_dispenser_origin.csv', index=False, encoding='utf-8-sig')

## Length Check

In [ ]:
import pandas as pd

df_original = pd.read_excel('./original_data.xlsx')
df_air = pd.read_csv('./water_dispenser_origin.csv')

mask = (
    # (df_original['components'] == '드라이기') |
    (df_original['components'] == '정수기')
)
df_filtered = df_original[mask]

df_filtered_count = len(df_filtered)
df_air_count = len(df_air)

print(df_filtered_count, df_air_count, df_filtered_count - df_air_count)

1629 1625 4


In [13]:
# 중복 여부까지 확인하기 위해 세 열을 합쳐서 비교 기준으로 사용
df_filtered['merge_key'] = df_filtered['ticket_id_hashed'].astype(str) + '|' + \
                           df_filtered['beforechange'].astype(str) + '|' + \
                           df_filtered['afterchange'].astype(str)

df_air['merge_key'] = df_air['ticket_id_hashed'].astype(str) + '|' + \
                      df_air['beforechange'].astype(str) + '|' + \
                      df_air['afterchange'].astype(str)

# df_filtered에만 있는 행 찾기
diff_row = df_filtered[~df_filtered['merge_key'].isin(df_air['merge_key'])]
diff_row.iloc[1]['afterchange']

/var/folders/k4/dp8jpl_52w1g8nsm7nqcr12m0000gn/T/ipykernel_2919/3316326408.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['merge_key'] = df_filtered['ticket_id_hashed'].astype(str) + '|' + \


'물량 설정시 100ml 까지 설정할 수 있으면 좋겠네요. 간단히 약 하나 먹을때 120은 항상 남네요'

In [14]:
len(diff_row)

4

In [15]:
diff_row_cleaned = diff_row.drop(columns=['merge_key'])
diff_row_cleaned.to_csv('./water_dispenser_diff.csv', index=False, encoding='utf-8-sig')

In [ ]:
diff_row_cleaned = diff_row.drop(columns=['merge_key'])

# diff_row['keyword']가 NaN인 경우 drop
diff_row_cleaned = diff_row_cleaned.dropna(subset=['keyword'])
len(diff_row_cleaned)

27

## I'm sorry Check

In [ ]:
concat_df = pd.read_csv('./water_dispenser_origin.csv')

In [ ]:
sorry_rows = water_dispenser_combined_df[water_dispenser_combined_df['generated_summary'].str.contains("I'm sorry", na=False)]
len(sorry_rows)

18

In [ ]:
# please, Please
please_rows = water_dispenser_combined_df[water_dispenser_combined_df['generated_summary'].str.contains("please", na=False)]
please_rows_2 = water_dispenser_combined_df[water_dispenser_combined_df['generated_summary'].str.contains("Please", na=False)]
len(please_rows), len(please_rows_2)

(11, 25)

In [ ]:
# concat 시 중복 제거
concat_sorry_please = pd.concat([sorry_rows, please_rows, please_rows_2], ignore_index=True)
concat_sorry_please = concat_sorry_please.drop_duplicates()
concat_sorry_please.to_csv('./water_dispenser_sorry_please.csv', index=False, encoding='utf-8-sig')

In [63]:
len(sorry_rows), len(please_rows), len(please_rows_2), len(concat_sorry_please)

(18, 11, 25, 41)

## Combine Missing One

In [ ]:
df_origin = pd.read_csv('./water_dispenser_origin.csv')
df_diff = pd.read_csv('./water_dispenser_diff.csv')

df_concat = pd.concat([df_origin, df_diff], ignore_index=True)
len(df_origin), len(df_diff), len(df_concat)

(3314, 1, 3315)

In [ ]:
df_concat.to_csv('./water_dispenser_origin.csv', index=False, encoding='utf-8-sig')

## Merge Re-I'm Sorry

In [ ]:
import pandas as pd

# 파일 경로
file_1st = './sorry2/water_dispenser_sorry_5_1st.csv'
file_2nd = './sorry2/water_dispenser_sorry_5_2nd.csv'
file_3rd = './sorry2/water_dispenser_sorry_5_3rd.csv'

# 데이터 읽기
df_1st = pd.read_csv(file_1st)
df_2nd = pd.read_csv(file_2nd)
df_3rd = pd.read_csv(file_3rd)

# 우선순위: 3rd > 2nd > 1st
# 각 데이터프레임에서 ticket_id와 generated_summary만 사용 (필요시 컬럼 수정)
key = 'ticket_id_hashed'
value = 'generated_summary'

# 1st를 기본으로 시작
df_final = df_1st.copy()

# 2nd의 해당 ticket_id에 대해 generated_summary만 업데이트
df_final.set_index(key, inplace=True)
df_2nd.set_index(key, inplace=True)
df_final.update(df_2nd)

# 3rd의 해당 ticket_id에 대해 generated_summary만 업데이트
df_3rd.set_index(key, inplace=True)
df_final.update(df_3rd)

# 인덱스 복구
df_final.reset_index(inplace=True)
len(df_final)

21

In [ ]:
# 결과 저장
df_final.to_csv('./sorry/water_dispenser_sorry_merged_5.csv', index=False)

In [ ]:
import pandas as pd

# 1. 원본 CSV 불러오기
origin_path = './water_dispenser_origin.csv'
merged_path = './sorry/water_dispenser_sorry_merged.csv'

df_origin = pd.read_csv(origin_path, encoding='utf-8-sig')
df_merged = pd.read_csv(merged_path, encoding='utf-8-sig')

print(len(df_origin), len(df_merged))

# key 컬럼 이름
key = 'ticket_id_hashed'
value = 'generated_summary'

df_merged_nodup = df_merged.drop_duplicates(subset=[key], keep='last')
df_origin = df_origin.drop_duplicates(subset=[key], keep='last')

# df_merged에 key와 value만 추출 (혹시 컬럼명이 다르면 맞게 수정)
df_origin.set_index(key, inplace=True)
df_merged_nodup.set_index(key, inplace=True)
df_origin.update(df_merged_nodup[[value]])

df_origin.reset_index(inplace=True)

# 2. 제거 대상 조건 설정
print(len(df_origin[df_origin['generated_summary'] == '-']))
print(len(df_origin[df_origin['keyword'].isna()]))

3348 32
15
0


In [ ]:
removed_df = df_origin[(df_origin['generated_summary'].isin(['-'])) | (df_origin['keyword'].isna())]
# removed_df = pd.concat([removed_df, diff_row_cleaned], ignore_index=True)

# 3. 정제된 데이터 설정
cleaned_df = df_origin[~df_origin['generated_summary'].isin(['-']) & ~df_origin['keyword'].isna()]

# 4. 확인 출력
print(f"원본 데이터 수: {len(df_origin)}")
print(f"제거된 데이터 수: {len(removed_df)}")
print(f"정제된 데이터 수: {len(cleaned_df)}")

# 5. 각각 저장
cleaned_df.to_csv("./water_dispenser_origin_updated.csv", index=False, encoding='utf-8-sig')
removed_df.to_csv("./water_dispenser_removed_rows.csv", index=False, encoding='utf-8-sig')

원본 데이터 수: 3348
제거된 데이터 수: 15
정제된 데이터 수: 3333
